In [1]:
import pandas as pd
from dash import Dash, html, dcc,dash_table, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc



In [2]:
electricity = pd.read_csv('electricity.csv')
app = Dash(external_stylesheets=[dbc.themes.SOLAR])
year_min = electricity['Year'].min()
year_max = electricity['Year'].max()



In [3]:



app.layout =  html.Div([
    html.H1("Electricity Prices by US State"),
    dcc.RangeSlider(id = 'year-slider',
                    min = year_min,
                    max = year_max,
                    value = [year_min,year_max],
                    marks= {i:str(i) for i in range(year_min,year_max+1)}
                    ),
    dcc.Graph(id = 'map-fig'),
    dash_table.DataTable(id = 'data-table'               )



    ]
)

@app.callback(
        Output('map-fig', 'figure'),
        Input('year-slider', 'value')
)
def map_update(selected_year):
    filtered_df = electricity[
        (electricity['Year'] >= selected_year[0]) &
        (electricity['Year'] <= selected_year[1])
    ]
    avg_price_electricity = filtered_df.groupby('US_State')['Residential Price'].mean().reset_index()

    map_fig = px.choropleth(avg_price_electricity,
                            locations='US_State',
                            locationmode='USA-states',
                            scope='usa',
                            color='Residential Price',
                            color_continuous_scale='reds')
    return map_fig

@app.callback(
    Output('data-table', 'data'),
    Input('map-fig', 'clickData'),
    Input('year-slider', 'value')
)

def update_table(clicked_data, selected_year):
    if clicked_data is None:
        return []
    us_state = clicked_data['points'][0]['location']
    filtered_df = electricity[
        (electricity['Year'] >= selected_year[0]) &
        (electricity['Year'] <= selected_year[1])&
        (electricity['US_State'] == us_state)]
    return filtered_df.to_dict('records')


if __name__ == '__main__':
    app.run(debug=True)
